In [36]:
!pip install --upgrade ipywidgets
!pip install segmentation-models-pytorch
!pip install kornia

Requirement already up-to-date: ipywidgets in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (8.0.2)
     |████████████████████████████████| 551 kB 5.4 MB/s eta 0:00:01


In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(".")
import numpy as np
from yaml import load, dump, Loader, Dumper
from tqdm import tqdm
import torch
import torchvision
from tabulate import tabulate

import argparse
import time

from competition_toolkit.dataloader import create_dataloader
from utils import create_run_dir, store_model_weights, record_scores

from competition_toolkit.eval_functions import calculate_score

In [2]:
opts = {
    'task': 2,
    'data_ratio': 1,
    'epochs': 20,
    'device': 0,
    'lr': 1e-3,
    'imagesize': 512,
    'rundir': 'runs',
    "task1": {
        "batchsize": 6,
        'shuffle': True,
    },
    "task2": {
        "batchsize": 4,
        'shuffle': True,
    }
}

In [8]:
import segmentation_models_pytorch as smp
from torch import nn

# model = smp.Linknet(
#         encoder_name="timm-resnest26d",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#         encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
#         in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#         classes=2,                      # model output channels (number of classes in your dataset)
#     )

trained_task1_net = torch.load("task1_0.8838_augs_3losses.pt")

model = nn.Sequential(
    nn.Conv2d(4, 8, 3, padding=1),
    nn.ReLU(),
    nn.Conv2d(8, 3, 3, padding=1),
    trained_task1_net
)

In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2


transforms = A.Compose(
    [
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.75),
        A.GaussNoise(p=0.2),
        ToTensorV2(),
    ]
)

image_only_transforms = A.Compose(
    [
        A.HueSaturationValue(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
        # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]
)

val_transforms = A.Compose(
    [
        ToTensorV2(),
    ]
)

val_image_only_transforms = None

In [18]:
import os

import numpy as np
from yaml import load, dump, Loader, Dumper
from tqdm import tqdm
import torch
import torchvision
from tabulate import tabulate

import argparse
import time

from competition_toolkit.dataloader import create_dataloader
from utils import create_run_dir, store_model_weights, record_scores

from competition_toolkit.eval_functions import calculate_score

from kornia.losses import DiceLoss
from boundary_loss import BoundaryLoss


def test(opts, dataloader, model, lossfn):
    model.eval()

    device = opts["device"]

    losstotal = np.zeros((len(dataloader)), dtype=float)
    ioutotal = np.zeros((len(dataloader)), dtype=float)
    bioutotal = np.zeros((len(dataloader)), dtype=float)
    scoretotal = np.zeros((len(dataloader)), dtype=float)

    for idx, batch in tqdm(enumerate(dataloader), leave=False, total=len(dataloader), desc="Test"):
        image, label, filename = batch
        image = image.to(device).float()
        label = label.to(device).long()

        output = model(image)#["out"]

        loss = lossfn(output, label).item()

        output = torch.argmax(torch.softmax(output, dim=1), dim=1)
        if device != "cpu":
            metrics = calculate_score(output.detach().cpu().numpy().astype(np.uint8),
                                      label.detach().cpu().numpy().astype(np.uint8))
        else:
            metrics = calculate_score(output.detach().numpy().astype(np.uint8), label.detach().numpy().astype(np.uint8))

        losstotal[idx] = loss
        ioutotal[idx] = metrics["iou"]
        bioutotal[idx] = metrics["biou"]
        scoretotal[idx] = metrics["score"]

    loss = round(losstotal.mean(), 4)
    iou = round(ioutotal.mean(), 4)
    biou = round(bioutotal.mean(), 4)
    score = round(scoretotal.mean(), 4)

    return loss, iou, biou, score


def train(model, opts, transforms, image_only_transforms, val_transforms, val_image_only_transforms):
    device = opts["device"]

    # The current model should be swapped with a different one of your choice
    # model = torchvision.models.segmentation.fcn_resnet50(pretrained=False, num_classes=opts["num_classes"])

    # if opts["task"] == 2:
    #     new_conv1 = torch.nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    #     model.backbone.conv1 = new_conv1

    model.to(device)
    # model = model.float()

    optimizer = torch.optim.Adam(model.parameters(), lr=opts["lr"])

    lossfn = torch.nn.CrossEntropyLoss()
    ce_loss = torch.nn.CrossEntropyLoss()
    dice_loss = DiceLoss(eps=1e-6)
    boundary_loss = BoundaryLoss()

    epochs = opts["epochs"]

    trainloader = create_dataloader(opts, transforms, image_only_transforms, "train")
    valloader = create_dataloader(opts, val_transforms, val_image_only_transforms, "validation")

    bestscore = 0

    for e in range(epochs):

        model.train()

        losstotal = np.zeros((len(trainloader)), dtype=float)
        scoretotal = np.zeros((len(trainloader)), dtype=float)
        ioutotal = np.zeros((len(trainloader)), dtype=float)
        bioutotal = np.zeros((len(trainloader)), dtype=float)

        stime = time.time()
        
        preview_loss = True
        for idx, batch in tqdm(enumerate(trainloader), leave=True, total=len(trainloader), desc="Train", position=0):
        # for idx, batch in enumerate(trainloader):
            image, label, filename = batch
            image = image.float().to(device)
            label = label.long().to(device).long()

            output = model(image)#["out"]
            bl = boundary_loss(output, label) * 0.1
            ce = ce_loss(output, label)
            di = dice_loss(output, label)
            loss = (bl + ce + di).mean()

            if preview_loss:
                print(e, "| bl, ce, di losses:", bl.item(), ce.item(), di.item())
                preview_loss = False

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            lossitem = loss.item()
            output = torch.argmax(torch.softmax(output, dim=1), dim=1)
            if device != "cpu":
                trainmetrics = calculate_score(output.detach().cpu().numpy().astype(np.uint8),
                                               label.detach().cpu().numpy().astype(np.uint8))
            else:
                trainmetrics = calculate_score(output.detach().numpy().astype(np.uint8),
                                               label.detach().numpy().astype(np.uint8))

            losstotal[idx] = lossitem
            ioutotal[idx] = trainmetrics["iou"]
            bioutotal[idx] = trainmetrics["biou"]
            scoretotal[idx] = trainmetrics["score"]

        testloss, testiou, testbiou, testscore = test(opts, valloader, model, lossfn)
        trainloss = round(losstotal.mean(), 4)
        trainiou = round(ioutotal.mean(), 4)
        trainbiou = round(bioutotal.mean(), 4)
        trainscore = round(scoretotal.mean(), 4)

        if testscore > bestscore:
            bestscore = testscore
            print("new best score:", bestscore, "- saving model weights")
            store_model_weights(opts, model, f"best", testscore, epoch=e)
        else:
            store_model_weights(opts, model, f"last", testscore, epoch=e)

        print("")
        print(tabulate(
            [["train", trainloss, trainiou, trainbiou, trainscore], ["test", testloss, testiou, testbiou, testscore]],
            headers=["Type", "Loss", "IoU", "BIoU", "Score"]))

        scoredict = {
            "epoch": e,
            "trainloss": trainloss,
            "testloss": testloss,
            "trainiou": trainiou,
            "testiou": testiou,
            "trainbiou": trainbiou,
            "testbiou": testbiou,
            "trainscore": trainscore,
            "testscore": testscore
        }

        record_scores(opts, scoredict)

In [19]:
# model.load_state_dict(torch.load("runs/best_task1_16_0.869100.pt"))
train(model, opts, transforms, image_only_transforms, val_transforms, val_image_only_transforms)

Reusing dataset mapai_training_data (/home/azureuser/.cache/huggingface/datasets/sjyhne___mapai_training_data/building_segmentation/1.0.0/b0b52f8c47ddbeae1962ab524cabb5fbed58d91cc70f9ac4c5981c071ad5f248)
Reusing dataset mapai_training_data (/home/azureuser/.cache/huggingface/datasets/sjyhne___mapai_training_data/building_segmentation/1.0.0/b0b52f8c47ddbeae1962ab524cabb5fbed58d91cc70f9ac4c5981c071ad5f248)
Train:  51%|█████▏    | 899/1750 [24:30<22:52,  1.61s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Train:  63%|██████▎   | 1110/1750 [27:10<15:34,  1.46s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the

Using number of images in traindataset: 7000/7000
Using number of images in validationdataset: 1500/1500
0 | bl, ce, di losses: 0.04499164596199989 0.07638144493103027 0.044584646821022034
Using number of images in traindataset: 7000/7000
Using number of images in validationdataset: 1500/1500
0 | bl, ce, di losses: 0.09742705523967743 0.47766655683517456 0.219547301530838
new best score: 0.8881 - saving model weights

Type      Loss     IoU    BIoU    Score
------  ------  ------  ------  -------
train   0.219   0.8512  0.7202   0.7857
test    0.0583  0.9265  0.8496   0.8881
1 | bl, ce, di losses: 0.03629662096500397 0.02051679790019989 0.010181158781051636
new best score: 0.8917 - saving model weights

Type      Loss     IoU    BIoU    Score
------  ------  ------  ------  -------
train   0.1811  0.893   0.7753   0.8341
test    0.0565  0.9295  0.8539   0.8917
3 | bl, ce, di losses: 0.03529854118824005 0.11649422347545624 0.04541245102882385
new best score: 0.8953 - saving model weight

KeyboardInterrupt: 

In [ ]:
lossfn = torch.nn.CrossEntropyLoss()

epochs = opts["epochs"]

valloader = create_dataloader(opts, val_transforms, val_image_only_transforms, "validation")

testloss, testiou, testbiou, testscore = test(opts, valloader, model, lossfn)
testloss, testiou, testbiou, testscore

In [69]:
# store_model_weights(opts, model, f"best", testscore, epoch="")
# torch.save(model, "task1_0.8838_augs_3losses.pt")
# torch.save(model, "task2_0.9014_augs_3losses.pt")

In [33]:
model.load_state_dict(torch.load("runs/best_task2_10_0.901400.pt"))
model.to(opts['device'])

Sequential(
  (0): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(8, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Linknet(
    (encoder): ResNestEncoder(
      (conv1): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (lay

In [21]:
lidar = load_lidar(lidarfilepath, (self.opts["imagesize"], self.opts["imagesize"]))

In [22]:
valloader = create_dataloader(opts, val_transforms, val_image_only_transforms, "validation")

Reusing dataset mapai_training_data (/home/azureuser/.cache/huggingface/datasets/sjyhne___mapai_training_data/building_segmentation/1.0.0/b0b52f8c47ddbeae1962ab524cabb5fbed58d91cc70f9ac4c5981c071ad5f248)


In [43]:
out = model.eval()([:1].to(opts['device']).float())
out.shape

In [ ]:
 calculate_score(output.detach().numpy().astype(np.uint8), label.detach().numpy().astype(np.uint8))

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(  out[0].detach().cpu().permute(1, 2, 0)[:,:,0]  )
plt.show()
plt.imshow(  out[0].detach().cpu().permute(1, 2, 0)[:,:,1]  )

# RESIZE TO 500?

In [32]:
lidar = load_lidar("../../data/train/lidar/6051_689_0.tif", (512, 512))

/tmp/ipykernel_96048/4045903770.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lidar = lidar.astype(np.float) / 255


In [19]:
!nvidia-smi

Sat Dec  3 17:22:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000001:00:00.0 Off |                  Off |
| N/A   42C    P0    26W /  70W |  15160MiB / 16127MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------